# dataset collection

In [ ]:
#!/usr/bin/env python

# Copyright (c) 2019 Computer Vision Center (CVC) at the Universitat Autonoma de
# Barcelona (UAB).
#
# This work is licensed under the terms of the MIT license.
# For a copy, see <https://opensource.org/licenses/MIT>.

import glob
import os
import sys


import cv2
import os

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

import argparse
import imageio # type: ignore
import random
import time
import logging
import numpy as np

video_writer = None
frame_count = 0

def main():
    global frame_count, video_writer

    argparser = argparse.ArgumentParser(
        description=__doc__)
    argparser.add_argument(
        '--host',
        metavar='H',
        default='127.0.0.1',
        help='IP of the host server (default: 127.0.0.1)')
    argparser.add_argument(
        '-p', '--port',
        metavar='P',
        default=2000,
        type=int,
        help='TCP port to listen to (default: 2000)')
    argparser.add_argument(
        '-n', '--number-of-vehicles',
        metavar='N',
        default=10,
        type=int,
        help='number of vehicles (default: 10)')
    argparser.add_argument(
        '-d', '--delay',
        metavar='D',
        default=2.0,
        type=float,
        help='delay in seconds between spawns (default: 2.0)')
    argparser.add_argument(
        '--rolename',
        metavar='NAME',
        default='hero',
        help='actor role name (default: "hero")')
    argparser.add_argument(
        '--safe',
        action='store_true',
        help='avoid spawning vehicles prone to accidents')
    argparser.add_argument(
        '-f', '--recorder_filename',
        metavar='F',
        default="test1.log",
        help='recorder filename (test1.log)')
    argparser.add_argument(
        '-t', '--recorder_time',
        metavar='T',
        default=0,
        type=int,
        help='recorder duration (auto-stop)')
    args = argparser.parse_args()

    actor_list = []
    logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.INFO)

    try:

        client = carla.Client(args.host, args.port)
        client.set_timeout(2.0)
        world = client.get_world()
        blueprints = world.get_blueprint_library().filter('vehicle.*')

        # ego_cam = None

        vehicles = world.get_actors().filter('vehicle.*')
        ego_vehicle = None
        for v in vehicles:
            if v.attributes.get('role_name') == 'hero':
                ego_vehicle = v
                break

        if ego_vehicle is None:
            print("No ego vehicle found! Please run manual_control.py first.")
            return
        
        if not os.path.exists('images_sept'):
            os.makedirs('images_sept')

        if not os.path.exists('videos'):
            os.makedirs('videos')
         
        frame_count = 0

        with open('datasept.csv', 'w') as f:
            f.write("frame,image_path,steer,throttle,brake,speed,heading,vehicle_id,vehicle_type\n")

        # === Setup Camera Sensor ===
        camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', '800')  # Image width
        camera_bp.set_attribute('image_size_y', '600')  # Image height
        camera_bp.set_attribute('fov', '120')            # Field of view

        #   Spawn camera at fixed location
        camera_transform = carla.Transform(carla.Location(x=-4.0, y=0.0, z=3.0), carla.Rotation(pitch=-5.0, yaw=0, roll=0))
        ego_cam = world.spawn_actor(camera_bp, camera_transform, attach_to=ego_vehicle)
       
        # Define callback to save images_sept in 'images_sept' folder
        def save_image(image):
            global frame_count, video_writer
            
            # Save image to 'images_sept' folder with incrementing filename
            filename = 'images_sept/%06d.png' % frame_count
            image.save_to_disk(filename)

            img = image.raw_data
            img = np.frombuffer(image.raw_data, dtype=np.uint8) 
            img = img.reshape((image.height, image.width, 4))  # BGRA format
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

            

      
            
            if video_writer is None:
                height, width, _ = img.shape
                
                output_video_path = 'videos/output_video.mp4'    # Output video file name
                fps = 30  

                video_writer = imageio.get_writer(output_video_path, fps=fps)
                # video_writer = True

                # if not video_writer.isOpened():
                #     logging.error(f"Failed to open video writer for {output_video_path}.")
                #     return
                
            
            print(f"Writing frame {frame_count}")
            

            video_writer.append_data(img)
            frame_count += 1

            control = ego_vehicle.get_control()
            
            # --- Extra info ---
            vel = ego_vehicle.get_velocity()
            speed = np.sqrt(vel.x**2 + vel.y**2 + vel.z**2)   # speed in m/s

            transform = ego_vehicle.get_transform()
            heading = transform.rotation.yaw                  # heading in degrees

            veh_id = ego_vehicle.id                           # numeric actor ID
            veh_type = ego_vehicle.type_id                    # e.g., vehicle.tesla.model3

            with open('datasept.csv', 'a') as f:
                    f.write(f"{frame_count},images_sept/{frame_count:06d}.png,"
            f"{control.steer},{control.throttle},{control.brake},"
            f"{speed:.2f},{heading:.2f},{veh_id},{veh_type}\n")
                    
        ego_cam.listen(lambda image: save_image(image))

        spawn_points = world.get_map().get_spawn_points()
        number_of_spawn_points = len(spawn_points)

        print('found %d spawn points.' % len(spawn_points))

        count = args.number_of_vehicles

        print("Recording on file: %s" % client.start_recorder(args.recorder_filename))

        # client.start_recorder('~/tutorial/recorder/recording01.log' or args.recorder_filename)


        if args.safe:
            blueprints = [x for x in blueprints if int(x.get_attribute('number_of_wheels')) == 4]
            blueprints = [x for x in blueprints if not x.id.endswith('microlino')]
            blueprints = [x for x in blueprints if not x.id.endswith('carlacola')]
            blueprints = [x for x in blueprints if not x.id.endswith('cybertruck')]
            blueprints = [x for x in blueprints if not x.id.endswith('t2')]
            blueprints = [x for x in blueprints if not x.id.endswith('sprinter')]
            blueprints = [x for x in blueprints if not x.id.endswith('firetruck')]
            blueprints = [x for x in blueprints if not x.id.endswith('ambulance')]

        # spawn_points = world.get_map().get_spawn_points()
        # number_of_spawn_points = len(spawn_points)

        if count < number_of_spawn_points:
            random.shuffle(spawn_points)
        elif count > number_of_spawn_points:
            msg = 'requested %d vehicles, but could only find %d spawn points'
            logging.warning(msg, count, number_of_spawn_points)
            count = number_of_spawn_points

        # @todo cannot import these directly.
        SpawnActor = carla.command.SpawnActor
        SetAutopilot = carla.command.SetAutopilot
        FutureActor = carla.command.FutureActor

        batch = []
        for n, transform in enumerate(spawn_points):
            if n >= count:
                break
            blueprint = random.choice(blueprints)
            if blueprint.has_attribute('color'):
                color = random.choice(blueprint.get_attribute('color').recommended_values)
                blueprint.set_attribute('color', color)
            blueprint.set_attribute('role_name', 'autopilot')
            batch.append(SpawnActor(blueprint, transform).then(SetAutopilot(FutureActor, True)))

        for response in client.apply_batch_sync(batch):
            if response.error:
                logging.error(response.error)
            else:
                actor_list.append(response.actor_id)

        print('spawned %d vehicles, press Ctrl+C to exit.' % len(actor_list))

        if (args.recorder_time > 0):
            time.sleep(args.recorder_time)
        else:
            while True:
                world.wait_for_tick()
                # time.sleep(0.1)
                

    finally:

        print('\ndestroying %d actors' % len(actor_list))
        client.apply_batch_sync([carla.command.DestroyActor(x) for x in actor_list])
        # camera.stop()
        # camera.destroy()

        print("Stop recording")

        client.stop_recorder()
        # if ego_cam is not None:
        #     ego_cam.destroy()
        #     print('Destroyed ego camera.')
  

        if video_writer is not None:
            video_writer.close()
            print('Video writer released.')


if __name__ == '__main__':

    try:
        main()
    except KeyboardInterrupt:
        pass
    finally:
        print('\ndone.')


# training code

In [ ]:
# train_fixed.py (enhanced for stability & reduced drift)
import os
import random
from collections import Counter
import numpy as np
import pandas as pd
from PIL import Image, ImageFilter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from torchvision.transforms import functional as TF
from torch.optim import AdamW
from torch.cuda.amp import GradScaler, autocast

# -------------------------
# Utilities & Config
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

CFG = {
    "img_h": 120,
    "img_w": 160,
    "batch_size": 16,            # smaller often more stable
    "epochs": 20,
    "lr": 3e-4,
    "weight_decay": 1e-5,
    "n_stack": 3,
    "use_speed": True,
    "balance_bins": 25,
    "max_grad_norm": 5.0,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "pin_memory": True,
}

# -------------------------
# Dataset (robust + jitter)
# -------------------------
class DrivingDataset(Dataset):
    def __init__(self, csv_file, img_root="", n_stack=1, augment=False, use_speed=True):
        self.df = pd.read_csv(csv_file)
        self.df.columns = self.df.columns.str.strip()
        self.img_root = img_root
        self.n_stack = max(1, n_stack)
        self.augment = augment
        self.use_speed = use_speed

        self.transform = transforms.Compose([
            transforms.Resize((CFG["img_h"], CFG["img_w"])),
            transforms.ToTensor(),
            transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5]),
        ])

        self.df = self.df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def _load(self, relpath):
        path = os.path.join(self.img_root, relpath) if self.img_root else relpath
        if not os.path.exists(path):
            return Image.new("RGB", (CFG["img_w"], CFG["img_h"]))
        return Image.open(path).convert("RGB")

    def _maybe_augment_single(self, img, steer):
        # brightness/contrast
        if torch.rand(1).item() < 0.5:
            img = TF.adjust_brightness(img, 0.8 + 0.4 * torch.rand(1).item())
            img = TF.adjust_contrast(img, 0.8 + 0.4 * torch.rand(1).item())
        # gamma (shadows)
        if torch.rand(1).item() < 0.3:
            img = TF.adjust_gamma(img, 0.7 + 0.6*torch.rand(1).item())
        # blur
        if torch.rand(1).item() < 0.2:
            img = img.filter(ImageFilter.GaussianBlur(radius=1))
        # translation
        if torch.rand(1).item() < 0.4:
            dx = int((torch.rand(1).item() * 2 - 1) * 8)
            img = TF.affine(img, angle=0.0, translate=(dx, 0), scale=1.0, shear=0.0)
            steer = float(steer) + (-dx / 8.0) * 0.12
        # flip
        if torch.rand(1).item() < 0.5:
            img = TF.hflip(img)
            steer = -float(steer)
        return img, float(steer)

    def __getitem__(self, idx):
        start = max(0, idx - (self.n_stack - 1))
        indices = list(range(start, idx + 1))
        while len(indices) < self.n_stack:
            indices.insert(0, indices[0])

        imgs = [ self._load(self.df.iloc[i]["image_path"]) for i in indices ]
        steer = float(self.df.iloc[idx]["steer"])
        throttle = float(self.df.iloc[idx]["throttle"]) if "throttle" in self.df.columns else 0.0
        brake = float(self.df.iloc[idx]["brake"]) if "brake" in self.df.columns else 0.0
        speed = float(self.df.iloc[idx]["speed"]) if "speed" in self.df.columns else 0.0

        if self.augment:
            imgs[0], steer = self._maybe_augment_single(imgs[0], steer)
            for j in range(1, len(imgs)):
                imgs[j] = imgs[0].copy()
            # inject steering noise (recovery training)
            if torch.rand(1).item() < 0.2:
                steer += np.random.normal(0, 0.05)

        tensors = [ self.transform(im) for im in imgs ]
        img_stack = torch.cat(tensors, dim=0)

        label = torch.tensor([steer, throttle, brake], dtype=torch.float32)
        # label smoothing / jitter
        if self.augment:
            label += 0.01*torch.randn_like(label)
            label = torch.clamp(label, min=-1.0, max=1.0)

        aux = torch.tensor([speed], dtype=torch.float32) if self.use_speed else torch.zeros(1, dtype=torch.float32)
        return img_stack, aux, label

# -------------------------
# Model
# -------------------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24,36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36,48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape, device=next(self.parameters()).device)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)

# -------------------------
# Helpers
# -------------------------
def make_steer_weights(df, n_bins=25):
    s = np.abs(df["steer"].values)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    inds = np.digitize(s, bins) - 1
    counts = Counter(inds)
    weights = np.array([1.0 / counts[i] for i in inds], dtype=np.float32)
    weights = weights / weights.mean()
    return weights

def speed_consistency_loss(pred, labels):
    target_accel = labels[:,1] - labels[:,2]
    pred_accel = pred[:,1] - pred[:,2]
    return F.mse_loss(pred_accel, target_accel)

# -------------------------
# Train / Validate
# -------------------------
def train_one_epoch(model, loader, opt, device, scaler=None, max_grad_norm=5.0,
                    loss_weights=(1.0, 0.6, 0.6)):
    model.train()
    total_loss = 0.0
    total_samples = 0
    for imgs, aux, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        aux = aux.to(device) if aux.nelement() != 0 else None

        opt.zero_grad()
        if scaler is not None:
            with autocast():
                out = model(imgs, aux)
                loss = (loss_weights[0]*F.mse_loss(out[:,0], labels[:,0]) +
                        loss_weights[1]*F.mse_loss(out[:,1], labels[:,1]) +
                        loss_weights[2]*F.mse_loss(out[:,2], labels[:,2]))
                loss += 0.2 * speed_consistency_loss(out, labels)
            scaler.scale(loss).backward()
            scaler.unscale_(opt)
            if max_grad_norm: torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(opt)
            scaler.update()
        else:
            out = model(imgs, aux)
            loss = (loss_weights[0]*F.mse_loss(out[:,0], labels[:,0]) +
                    loss_weights[1]*F.mse_loss(out[:,1], labels[:,1]) +
                    loss_weights[2]*F.mse_loss(out[:,2], labels[:,2]))
            loss += 0.2 * speed_consistency_loss(out, labels)
            loss.backward()
            if max_grad_norm: torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            opt.step()

        total_loss += loss.item() * imgs.size(0)
        total_samples += imgs.size(0)
    return total_loss / total_samples

def validate(model, loader, device):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for imgs, aux, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            aux = aux.to(device) if aux.nelement() != 0 else None
            out = model(imgs, aux)
            loss = F.mse_loss(out, labels, reduction="mean")
            loss += 0.2 * speed_consistency_loss(out, labels)
            total_loss += loss.item() * imgs.size(0)
            total_samples += imgs.size(0)
    return total_loss / total_samples

# -------------------------
# Main
# -------------------------
def main(train_csv="train.csv", val_csv="val.csv", img_root=""):
    device = torch.device(CFG["device"])
    print("Device:", device)

    train_df = pd.read_csv(train_csv)
    weights = make_steer_weights(train_df, n_bins=CFG["balance_bins"])
    sampler = WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

    train_ds = DrivingDataset(train_csv, img_root=img_root, n_stack=CFG["n_stack"], augment=True, use_speed=CFG["use_speed"])
    val_ds = DrivingDataset(val_csv, img_root=img_root, n_stack=CFG["n_stack"], augment=False, use_speed=CFG["use_speed"])

    num_workers = 4 if device.type == "cuda" else 0
    pin_memory = True if device.type == "cuda" else False

    train_loader = DataLoader(train_ds, batch_size=CFG["batch_size"], sampler=sampler,
                              num_workers=num_workers, pin_memory=pin_memory)
    val_loader = DataLoader(val_ds, batch_size=CFG["batch_size"], shuffle=False,
                            num_workers=max(0, num_workers//2), pin_memory=pin_memory)

    model = DrivingNet(in_ch=3, n_stack=CFG["n_stack"], use_speed=CFG["use_speed"]).to(device)
    model.initialize((3*CFG["n_stack"], CFG["img_h"], CFG["img_w"]))
    model = model.to(device)   # <-- move everything after building fc/head

    opt = AdamW(model.parameters(), lr=CFG["lr"], weight_decay=CFG["weight_decay"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)
    scaler = GradScaler() if device.type == "cuda" else None

    best = float("inf")
    os.makedirs("checkpoints", exist_ok=True)

    for epoch in range(1, CFG["epochs"]+1):
        tr_loss = train_one_epoch(model, train_loader, opt, device, scaler=scaler,
                                  max_grad_norm=CFG["max_grad_norm"])
        val_loss = validate(model, val_loader, device)
        scheduler.step(val_loss)
        print(f"[{epoch}] train_loss={tr_loss:.6f}  val_loss={val_loss:.6f}")

        torch.save({"epoch": epoch, "state": model.state_dict(), "opt": opt.state_dict()},
                   f"checkpoints/ckpt_epoch{epoch:02d}.pth")
        if val_loss < best:
            best = val_loss
            torch.save(model.state_dict(), "checkpoints/new_best_model.pth")
            print(" -> saved new_best_model.pth")

    print("Done. best val:", best)

if __name__ == "__main__":
    main()


# evaluation- drift code

In [1]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import signal
import math

# Environment setup
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24,36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36,48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape, device=next(self.parameters()).device)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    """
    Preprocess a stack of 3 RGB frames for the model.
    Matches the exact preprocessing used during training.
    """
    if len(frame_stack) != 3:
        return None
    
    processed_frames = []
    
    # Process each frame individually first (as they would be during training)
    for frame in frame_stack:
        # Resize if needed (should already be 120x160)
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        
        # Convert to float32 and normalize to [0, 1]
        frame = frame.astype(np.float32) / 255.0
        
        # Normalize to [-1, 1] range (standard ImageNet-style normalization)
        frame = (frame - 0.5) / 0.5
        
        processed_frames.append(frame)
    
    # Stack the processed frames along channel dimension
    # This creates a (120, 160, 9) array
    stacked = np.concatenate(processed_frames, axis=2)
    
    # Convert to CHW format for PyTorch (9, 120, 160)
    stacked = np.transpose(stacked, (2, 0, 1))
    
    # Convert to tensor and add batch dimension
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)  # (1, 9, 120, 160)
    
    return tensor.to(device)


class ImprovedPIDController:
    """Enhanced PID controller with better tuning and safeguards"""
    
    def __init__(self):
        # PID gains - tuned for better lane keeping
        self.Kp = 0.8  # Increased proportional gain
        self.Ki = 0.02  # Reduced integral gain to prevent windup
        self.Kd = 0.15  # Increased derivative gain for stability
        
        # State variables
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        
        # Anti-windup limits
        self.integral_limit = 0.5
        self.output_limit = 0.6  # Reduced steering limit
        
    def update(self, error, current_time):
        """Update PID controller with current error"""
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05  # Default dt
        else:
            dt = max(current_time - self.prev_time, 0.001)  # Prevent division by zero
            self.prev_time = current_time
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term with anti-windup
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        
        # Derivative term with smoothing
        if dt > 0:
            derivative = (error - self.prev_error) / dt
            D = self.Kd * derivative
        else:
            D = 0.0
        
        self.prev_error = error
        
        # Combine and limit output
        output = P + I + D
        return np.clip(output, -self.output_limit, self.output_limit)
    
    def reset(self):
        """Reset PID controller state"""
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Enhanced control system
        self.pid_controller = ImprovedPIDController()
        self.throttle_smooth = 0.3
        self.steer_smooth = 0.0
        
        # Control smoothing parameters
        self.steer_alpha = 0.3  # Steering smoothing factor
        self.throttle_alpha = 0.2  # Throttle smoothing factor
        
        # Lane keeping parameters
        self.target_speed_kmh = 30  # Target speed in km/h
        self.max_lane_offset = 2.0  # Maximum allowed lane offset before correction
        
        # Statistics
        self.collision_count = 0
        self.collision_times = []
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate lateral error from lane center using CARLA waypoints"""
        try:
            # Get the nearest waypoint
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if waypoint is None:
                return 0.0
            
            # Calculate lateral offset
            lane_center = waypoint.transform.location
            
            # Vector from lane center to vehicle
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            
            # Get lane direction (perpendicular to road direction)
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            
            # Calculate cross-track error (positive = right of center, negative = left)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Calculate heading error (difference between vehicle and lane heading)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            
            # Normalize heading error to [-pi, pi]
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combine cross-track and heading errors
            # Weight heading error more heavily for better stability
            total_error = cross_track_error + 0.5 * heading_error
            
            return total_error
            
        except Exception as e:
            print(f"Error calculating lane error: {e}")
            return 0.0

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        """Adaptive speed control based on lane position and steering input"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed when far from lane center or steering heavily
        offset_factor = max(0.7, 1.0 - abs(lane_offset) / 3.0)  # Slow down if offset > 1.5m
        steering_factor = max(0.8, 1.0 - abs(steering_magnitude) / 0.5)  # Slow down during sharp turns
        
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        
        # Simple speed controller
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.2, 0.4 + speed_error * 0.2)
            
        return throttle

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown initiated...")
        self.running = False
        
    def connect_carla(self):
        """Connect to CARLA simulator"""
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.map = self.world.get_map()
        
        # Set synchronous mode for stable evaluation
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05  # 20 FPS
        self.world.apply_settings(settings)
        
    def spawn_vehicle(self):
        """Spawn vehicle at random location"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Use a standard vehicle
        vehicle_bp = blueprint_library.filter("vehicle.mercedes.coupe_2020")[0]
        if not vehicle_bp:
            vehicle_bp = blueprint_library.filter("vehicle.*")[0]
        
        # Get spawn point
        spawn_points = self.map.get_spawn_points()
        spawn_point = random.choice(spawn_points)
        spawn_point.location.z += 0.5
        
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        print(f"Spawned vehicle at: ({spawn_point.location.x:.2f}, {spawn_point.location.y:.2f})")
        
        return spawn_point.location
        
    def setup_sensors(self):
        """Setup camera and collision sensors"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Camera sensor
        camera_bp = blueprint_library.find("sensor.camera.rgb")
        camera_bp.set_attribute("image_size_x", "160")
        camera_bp.set_attribute("image_size_y", "120")
        camera_bp.set_attribute("fov", "110")
        
        camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
        self.camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=self.vehicle)
        
        # Collision sensor
        collision_bp = blueprint_library.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        
        # Setup callbacks
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        
        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)
        
    def camera_callback(self, image):
        """Process camera images"""
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = array.reshape((image.height, image.width, 4))[:, :, :3]  # BGRA -> BGR
        array = array[:, :, ::-1]  # BGR -> RGB
        self.latest_image = array
        self.new_image_available = True
        
    def collision_callback(self, event):
        """Handle collision events"""
        self.collision_count += 1
        collision_time = time.time() - self.start_time
        self.collision_times.append(collision_time)
        print(f"\n*** Collision #{self.collision_count} at {collision_time:.2f}s ***")
        print(f"    Impact with: {event.other_actor.type_id}")
        self.collision_occurred = True

        if self.collision_occurred:
            # Reset speed to 0 to stop the vehicle
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
            
            # Reset PID controller
            self.pid_controller.reset()
            
            # Neutralize steering for recovery
            control = carla.VehicleControl()
            control.steer = 0.0
            self.vehicle.apply_control(control)
            
            # Wait before continuing
            time.sleep(1)
            self.collision_occurred = False

    def load_model(self):
        """Load the trained model"""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Initialize model
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True).to(self.device)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device) 
        
        # Load weights
        if os.path.exists(self.model_path):
            checkpoint = torch.load(self.model_path, map_location=self.device, weights_only=False)
            self.model.load_state_dict(checkpoint)
            print(f"Model loaded successfully from {self.model_path}")
        else:
            raise FileNotFoundError(f"Model checkpoint not found at {self.model_path}")
            
        self.model.eval()
        
    def run_evaluation(self):
        """Main evaluation loop with enhanced lane keeping"""
        print("\n" + "="*60)
        print("ENHANCED CLOSED-LOOP EVALUATION STARTED")
        print("="*60)
        print("Press Ctrl+C to stop evaluation gracefully\n")
        
        # Initialize
        self.connect_carla()
        start_location = self.spawn_vehicle()
        self.setup_sensors()
        self.load_model()
        
        # Setup spectator
        spectator = self.world.get_spectator()
        
        # Initialize control
        control = carla.VehicleControl()
        control.manual_gear_shift = False
        
        # Wait for initial frames
        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)
        
        # Initialize variables
        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log_time = time.time()
        
        # Open log file
        log_filename = f"enhanced_evaluation_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,distance_traveled\n")
        
        print("Starting enhanced autonomous driving...\n")
        
        try:
            while self.running and not self.collision_occurred:
                # Update frame stack
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False
                
                # Process when we have 3 frames
                if len(frame_stack) == 3:
                    # Get vehicle state
                    transform = self.vehicle.get_transform()
                    location = transform.location
                    rotation = transform.rotation
                    
                    # Calculate speed
                    velocity = self.vehicle.get_velocity()
                    speed_mps = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
                    speed_kmh = speed_mps * 3.6
                    
                    # Calculate lane error
                    lane_error = self.calculate_lane_error(location, rotation)
                    
                    # Preprocess frames for model
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    
                    if input_tensor is not None:
                        # Prepare speed input
                        speed_normalized = min(speed_mps / 30.0, 1.0)
                        speed_tensor = torch.tensor([[speed_normalized]], dtype=torch.float32, device=self.device)
                        
                        # Model inference
                        with torch.no_grad():
                            output = self.model(input_tensor, aux=speed_tensor)
                        
                        # Extract model outputs
                        model_steer, model_throttle, model_brake = output[0].cpu().numpy()
                        
                        # Enhanced steering control combining model output and lane keeping
                        current_time = time.time()
                        
                        # Use PID controller to correct lane error
                        pid_correction = self.pid_controller.update(lane_error, current_time)
                        
                        # Combine model steering with PID correction
                        # Give more weight to PID when lane error is large
                        error_weight = min(abs(lane_error) / 1.0, 0.8)  # Max 80% PID weight
                        combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                        
                        # Apply steering smoothing
                        final_steer = (self.steer_alpha * combined_steer + 
                                     (1 - self.steer_alpha) * self.steer_smooth)
                        self.steer_smooth = final_steer
                        
                        # Clamp steering to safe limits
                        final_steer = np.clip(final_steer, -0.6, 0.6)
                        
                        # Adaptive throttle control
                        adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                        
                        # Apply throttle smoothing
                        final_throttle = (self.throttle_alpha * adaptive_throttle + 
                                        (1 - self.throttle_alpha) * self.throttle_smooth)
                        self.throttle_smooth = final_throttle
                        
                        # Ensure minimum throttle to prevent stalling
                        final_throttle = max(final_throttle, 0.2)
                        final_throttle = min(final_throttle, 0.8)
                        
                        # Apply controls
                        control.steer = float(final_steer)
                        control.throttle = float(final_throttle)
                        control.brake = 0.0
                        control.hand_brake = False
                        self.vehicle.apply_control(control)
                    
                        # Calculate metrics
                        distance_traveled = location.distance(start_location)
                        
                        # Store data
                        self.deviations.append(abs(lane_error))
                        self.controls_history.append({
                            'frame': frame_count,
                            'model_steer': model_steer,
                            'final_steer': final_steer,
                            'throttle': final_throttle,
                            'speed': speed_mps,
                            'lane_error': lane_error
                        })
                        self.trajectory.append({
                            'x': location.x,
                            'y': location.y,
                            'z': location.z,
                            'yaw': rotation.yaw
                        })
                        
                        # Log to file
                        timestamp = time.time() - self.start_time
                        log_file.write(
                            f"{timestamp:.3f},{frame_count},{location.x:.2f},{location.y:.2f},{location.z:.2f},"
                            f"{rotation.yaw:.2f},{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},"
                            f"{final_steer:.4f},{final_throttle:.4f},{lane_error:.4f},{distance_traveled:.2f}\n"
                        )
                        log_file.flush()
                        
                        # Update spectator view
                        backward_vec = transform.get_forward_vector() * -8
                        spectator_loc = location + backward_vec
                        spectator_loc.z += 4
                        spectator_rot = carla.Rotation(pitch=-20, yaw=rotation.yaw, roll=0)
                        spectator.set_transform(carla.Transform(spectator_loc, spectator_rot))
                        
                        # Print status every second
                        current_time = time.time()
                        if current_time - last_log_time >= 1.0:
                            print(f"[{timestamp:6.1f}s] Frame {frame_count:5d} | "
                                  f"Speed: {speed_kmh:5.1f} km/h | "
                                  f"Distance: {distance_traveled:6.1f}m | "
                                  f"Lane error: {lane_error:+.3f}m | "
                                  f"Steering (M/F): {model_steer:+.3f}/{final_steer:+.3f}")
                            last_log_time = current_time
                    
                    frame_count += 1
                
                # Tick the simulation
                self.world.tick()

        except Exception as e:
            print(f"\nError during evaluation: {e}")
            
        finally:
            # Close log file
            log_file.close()
            
            # Print final statistics
            self.print_statistics(frame_count)
            
            # Cleanup
            self.cleanup()

    def print_statistics(self, frame_count):
        """Print enhanced evaluation statistics"""
        elapsed_time = time.time() - self.start_time
        
        print("\n" + "="*60)
        print("ENHANCED EVALUATION STATISTICS")
        print("="*60)
        
        print(f"Total frames processed: {frame_count}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        print(f"Average FPS: {frame_count/elapsed_time:.2f}")
        print(f"Total collisions: {self.collision_count}")
        
        if len(self.deviations) > 0:
            print(f"\nLane keeping performance:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} meters")
            print(f"  Std deviation: {np.std(self.deviations):.3f} meters")
            print(f"  Max offset: {np.max(self.deviations):.3f} meters")
            print(f"  95th percentile: {np.percentile(self.deviations, 95):.3f} meters")
            
            # Calculate percentage of time within lane bounds
            within_lane = np.sum(np.array(self.deviations) <= 1.5) / len(self.deviations) * 100
            print(f"  Time within 1.5m of lane center: {within_lane:.1f}%")
            
        if len(self.trajectory) > 0:
            first_pos = self.trajectory[0]
            last_pos = self.trajectory[-1]
            total_distance = np.sqrt(
                (last_pos['x'] - first_pos['x'])**2 + 
                (last_pos['y'] - first_pos['y'])**2
            )
            print(f"\nTotal distance traveled: {total_distance:.2f} meters")
            
        if len(self.controls_history) > 0:
            all_speeds = [c['speed'] for c in self.controls_history]
            all_lane_errors = [c['lane_error'] for c in self.controls_history]
            
            print(f"\nSpeed statistics:")
            print(f"  Average speed: {np.mean(all_speeds)*3.6:.2f} km/h")
            print(f"  Max speed: {np.max(all_speeds)*3.6:.2f} km/h")
            
            print(f"\nLane error statistics:")
            print(f"  Mean absolute error: {np.mean(np.abs(all_lane_errors)):.3f} meters")
            print(f"  RMS error: {np.sqrt(np.mean(np.array(all_lane_errors)**2)):.3f} meters")
                
        print("="*60)
        
    def cleanup(self):
        """Clean up CARLA actors and reset settings"""
        print("\nCleaning up...")
        
        # Reset to asynchronous mode
        if hasattr(self, 'world'):
            settings = self.world.get_settings()
            settings.synchronous_mode = False
            self.world.apply_settings(settings)
        
        # Destroy actors
        actors_to_destroy = []
        if hasattr(self, 'camera'):
            self.camera.stop()
            actors_to_destroy.append(self.camera)
        if hasattr(self, 'collision_sensor'):
            self.collision_sensor.stop()
            actors_to_destroy.append(self.collision_sensor)
        if hasattr(self, 'vehicle'):
            actors_to_destroy.append(self.vehicle)
            
        for actor in actors_to_destroy:
            if actor is not None:
                actor.destroy()
                
        print("Cleanup complete.")


def main():
    # Path to your trained model
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    # Create evaluator and run
    evaluator = ClosedLoopEvaluator(model_path=MODEL_PATH)
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()


ENHANCED CLOSED-LOOP EVALUATION STARTED
Press Ctrl+C to stop evaluation gracefully

Spawned vehicle at: (-87.28, 24.44)
Using device: cuda
Model loaded successfully from /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming up sensors...
Starting enhanced autonomous driving...

[   1.0s] Frame    45 | Speed:  14.7 km/h | Distance:    1.7m | Lane error: -0.008m | Steering (M/F): +0.002/+0.006
[   2.0s] Frame    95 | Speed:  25.4 km/h | Distance:   14.9m | Lane error: -0.217m | Steering (M/F): +0.006/-0.034
[   3.0s] Frame   148 | Speed:  27.1 km/h | Distance:   34.5m | Lane error: -0.140m | Steering (M/F): +0.028/-0.012
[   4.0s] Frame   199 | Speed:  25.0 km/h | Distance:   53.4m | Lane error: +0.051m | Steering (M/F): -0.003/-0.001
[   5.0s] Frame   249 | Speed:  28.3 km/h | Distance:   71.8m | Lane error: +0.190m | Steering (M/F): -0.004/+0.035
[   6.0s] Frame   302 | Speed:  24.2 km/h | Distance:   91.2m | Lane error: -0.079m | Steering (M/F): -0.00

## success-closed loop video

In [1]:
import carla
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
from PIL import Image
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math

# Environment setup
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24,36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36,48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape, device=next(self.parameters()).device)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    """
    Preprocess a stack of 3 RGB frames for the model.
    Matches the exact preprocessing used during training.
    """
    if len(frame_stack) != 3:
        return None
    
    processed_frames = []
    
    # Process each frame individually first (as they would be during training)
    for frame in frame_stack:
        # Resize if needed (should already be 120x160)
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        
        # Convert to float32 and normalize to [0, 1]
        frame = frame.astype(np.float32) / 255.0
        
        # Normalize to [-1, 1] range (standard ImageNet-style normalization)
        frame = (frame - 0.5) / 0.5
        
        processed_frames.append(frame)
    
    # Stack the processed frames along channel dimension
    # This creates a (120, 160, 9) array
    stacked = np.concatenate(processed_frames, axis=2)
    
    # Convert to CHW format for PyTorch (9, 120, 160)
    stacked = np.transpose(stacked, (2, 0, 1))
    
    # Convert to tensor and add batch dimension
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)  # (1, 9, 120, 160)
    
    return tensor.to(device)


class ImprovedPIDController:
    """Enhanced PID controller with better tuning and safeguards"""
    
    def __init__(self):
        # PID gains - tuned for better lane keeping
        self.Kp = 0.8  # Increased proportional gain
        self.Ki = 0.02  # Reduced integral gain to prevent windup
        self.Kd = 0.15  # Increased derivative gain for stability
        
        # State variables
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        
        # Anti-windup limits
        self.integral_limit = 0.5
        self.output_limit = 0.6  # Reduced steering limit
        
    def update(self, error, current_time):
        """Update PID controller with current error"""
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05  # Default dt
        else:
            dt = max(current_time - self.prev_time, 0.001)  # Prevent division by zero
            self.prev_time = current_time
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term with anti-windup
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        
        # Derivative term with smoothing
        if dt > 0:
            derivative = (error - self.prev_error) / dt
            D = self.Kd * derivative
        else:
            D = 0.0
        
        self.prev_error = error
        
        # Combine and limit output
        output = P + I + D
        return np.clip(output, -self.output_limit, self.output_limit)
    
    def reset(self):
        """Reset PID controller state"""
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Enhanced control system
        self.pid_controller = ImprovedPIDController()
        self.throttle_smooth = 0.3
        self.steer_smooth = 0.0
        
        # Control smoothing parameters
        self.steer_alpha = 0.3  # Steering smoothing factor
        self.throttle_alpha = 0.2  # Throttle smoothing factor
        
        # Lane keeping parameters
        self.target_speed_kmh = 15  # Target speed in km/h
        self.max_lane_offset = 2.0  # Maximum allowed lane offset before correction
        
        # Statistics
        self.collision_count = 0
        self.collision_times = []
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate lateral error from lane center using CARLA waypoints"""
        try:
            # Get the nearest waypoint
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if waypoint is None:
                return 0.0
            
            # Calculate lateral offset
            lane_center = waypoint.transform.location
            
            # Vector from lane center to vehicle
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            
            # Get lane direction (perpendicular to road direction)
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            
            # Calculate cross-track error (positive = right of center, negative = left)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Calculate heading error (difference between vehicle and lane heading)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            
            # Normalize heading error to [-pi, pi]
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combine cross-track and heading errors
            # Weight heading error more heavily for better stability
            total_error = cross_track_error + 0.5 * heading_error
            
            return total_error
            
        except Exception as e:
            print(f"Error calculating lane error: {e}")
            return 0.0

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        """Adaptive speed control based on lane position and steering input"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed when far from lane center or steering heavily
        offset_factor = max(0.7, 1.0 - abs(lane_offset) / 3.0)  # Slow down if offset > 1.5m
        steering_factor = max(0.8, 1.0 - abs(steering_magnitude) / 0.5)  # Slow down during sharp turns
        
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        
        # Simple speed controller
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.2, 0.4 + speed_error * 0.2)
            
        return throttle

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown initiated...")
        self.running = False
        
    def connect_carla(self):
        """Connect to CARLA simulator"""
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        # self.world = self.client.load_world("Town05")
        self.map = self.world.get_map()
        
        # Set synchronous mode for stable evaluation
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05  # 20 FPS
        self.world.apply_settings(settings)
        
    def spawn_vehicle(self):
        """Spawn vehicle at random location"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Use a standard vehicle
        vehicle_bp = blueprint_library.filter("vehicle.mercedes.coupe_2020")[0]
        if not vehicle_bp:
            vehicle_bp = blueprint_library.filter("vehicle.*")[0]
        
        # Get spawn point
        spawn_points = self.map.get_spawn_points()
        spawn_point = random.choice(spawn_points)
        spawn_point.location.z += 0.5
        
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        print(f"Spawned vehicle at: ({spawn_point.location.x:.2f}, {spawn_point.location.y:.2f})")
        
        return spawn_point.location
        
    def setup_sensors(self):
        """Setup camera and collision sensors"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Camera sensor (set exactly like CNN script: 800x600, fov 120, camera at x=-4 pitch=-5)
        camera_bp = blueprint_library.find("sensor.camera.rgb")
        camera_bp.set_attribute("image_size_x", "800")
        camera_bp.set_attribute("image_size_y", "600")
        camera_bp.set_attribute("fov", "120")
        
        camera_transform = carla.Transform(carla.Location(x=-4, y=0, z=2), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=self.vehicle)
        
        # Collision sensor
        collision_bp = blueprint_library.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        
        # Setup callbacks
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        
        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)
        
    def camera_callback(self, image):
        """Process camera images (match CNN script format: RGB)"""
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = array.reshape((image.height, image.width, 4))[:, :, :3]  # BGRA -> BGR
        array = array[:, :, ::-1]  # BGR -> RGB
        self.latest_image = array
        self.new_image_available = True
        
    def collision_callback(self, event):
        """Handle collision events"""
        self.collision_count += 1
        collision_time = time.time() - self.start_time
        self.collision_times.append(collision_time)
        print(f"\n*** Collision #{self.collision_count} at {collision_time:.2f}s ***")
        print(f"    Impact with: {event.other_actor.type_id}")
        self.collision_occurred = True

        if self.collision_occurred:
            # Reset speed to 0 to stop the vehicle
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
            
            # Reset PID controller
            self.pid_controller.reset()
            
            # Neutralize steering for recovery
            control = carla.VehicleControl()
            control.steer = 0.0
            self.vehicle.apply_control(control)
            
            # Wait before continuing
            time.sleep(1)
            self.collision_occurred = False

    def load_model(self):
        """Load the trained model"""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Initialize model
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True).to(self.device)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device) 
        
        # Load weights
        if os.path.exists(self.model_path):
            checkpoint = torch.load(self.model_path, map_location=self.device, weights_only=False)
            self.model.load_state_dict(checkpoint)
            print(f"Model loaded successfully from {self.model_path}")
        else:
            raise FileNotFoundError(f"Model checkpoint not found at {self.model_path}")
            
        self.model.eval()
        
    def run_evaluation(self):
        """Main evaluation loop with enhanced lane keeping"""
        print("\n" + "="*60)
        print("ENHANCED CLOSED-LOOP EVALUATION STARTED")
        print("="*60)
        print("Press Ctrl+C to stop evaluation gracefully\n")
        
        # Initialize
        self.connect_carla()
        start_location = self.spawn_vehicle()
        self.setup_sensors()
        self.load_model()
        
        # Setup spectator
        spectator = self.world.get_spectator()
        
        # Initialize control
        control = carla.VehicleControl()
        control.manual_gear_shift = False
        
        # Wait for initial frames
        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)
        
        # Initialize variables
        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log_time = time.time()
        
        # Open log file
        log_filename = f"enhanced_evaluation_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,distance_traveled\n")
        
        # Image directory for video frames (using CNN-style naming)
        image_dir = f"enhanced_eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)
        
        print("Starting enhanced autonomous driving...\n")
        
        try:
            while self.running and not self.collision_occurred:
                # Update frame stack
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False
                
                # Process when we have 3 frames
                if len(frame_stack) == 3:
                    # Get vehicle state
                    transform = self.vehicle.get_transform()
                    location = transform.location
                    rotation = transform.rotation
                    
                    # Calculate speed
                    velocity = self.vehicle.get_velocity()
                    speed_mps = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
                    speed_kmh = speed_mps * 3.6
                    
                    # Calculate lane error
                    lane_error = self.calculate_lane_error(location, rotation)
                    
                    # Preprocess frames for model
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    
                    if input_tensor is not None:
                        # Prepare speed input
                        speed_normalized = min(speed_mps / 30.0, 1.0)
                        speed_tensor = torch.tensor([[speed_normalized]], dtype=torch.float32, device=self.device)
                        
                        # Model inference
                        with torch.no_grad():
                            output = self.model(input_tensor, aux=speed_tensor)
                        
                        # Extract model outputs
                        model_steer, model_throttle, model_brake = output[0].cpu().numpy()
                        
                        # Enhanced steering control combining model output and lane keeping
                        current_time = time.time()
                        
                        # Use PID controller to correct lane error
                        pid_correction = self.pid_controller.update(lane_error, current_time)
                        
                        # Combine model steering with PID correction
                        # Give more weight to PID when lane error is large
                        error_weight = min(abs(lane_error) / 1.0, 0.8)  # Max 80% PID weight
                        combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                        
                        # Apply steering smoothing
                        final_steer = (self.steer_alpha * combined_steer + 
                                     (1 - self.steer_alpha) * self.steer_smooth)
                        self.steer_smooth = final_steer
                        
                        # Clamp steering to safe limits
                        final_steer = np.clip(final_steer, -0.6, 0.6)
                        
                        # Adaptive throttle control
                        adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                        
                        # Apply throttle smoothing
                        final_throttle = (self.throttle_alpha * adaptive_throttle + 
                                        (1 - self.throttle_alpha) * self.throttle_smooth)
                        self.throttle_smooth = final_throttle
                        
                        # Ensure minimum throttle to prevent stalling
                        final_throttle = max(final_throttle, 0.2)
                        final_throttle = min(final_throttle, 0.8)
                        
                        # Apply controls
                        control.steer = float(final_steer)
                        control.throttle = float(final_throttle)
                        control.brake = 0.0
                        control.hand_brake = False
                        self.vehicle.apply_control(control)
                        
                        # Save frame exactly like CNN script
                        try:
                            frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(frame_filename, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                        except Exception as e:
                            print(f"Warning: failed to save frame {frame_count}: {e}")
                    
                        # Calculate metrics
                        distance_traveled = location.distance(start_location)
                        
                        # Store data
                        self.deviations.append(abs(lane_error))
                        self.controls_history.append({
                            'frame': frame_count,
                            'model_steer': model_steer,
                            'final_steer': final_steer,
                            'throttle': final_throttle,
                            'speed': speed_mps,
                            'lane_error': lane_error
                        })
                        self.trajectory.append({
                            'x': location.x,
                            'y': location.y,
                            'z': location.z,
                            'yaw': rotation.yaw
                        })
                        
                        # Log to file
                        timestamp = time.time() - self.start_time
                        log_file.write(
                            f"{timestamp:.3f},{frame_count},{location.x:.2f},{location.y:.2f},{location.z:.2f},"
                            f"{rotation.yaw:.2f},{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},"
                            f"{final_steer:.4f},{final_throttle:.4f},{lane_error:.4f},{distance_traveled:.2f}\n"
                        )
                        log_file.flush()
                        
                        # Update spectator view (use CNN-style spectator follow: -6 and z=3, pitch=-15)
                        transform = self.vehicle.get_transform()
                        loc = transform.location
                        rot = transform.rotation
                        spectator.set_transform(carla.Transform(loc + transform.get_forward_vector()*-6 + carla.Location(z=3),
                                                                carla.Rotation(pitch=-15, yaw=rot.yaw)))
                        
                        # Print status every second
                        current_time = time.time()
                        if current_time - last_log_time >= 1.0:
                            print(f"[{timestamp:6.1f}s] Frame {frame_count:5d} | "
                                  f"Speed: {speed_kmh:5.1f} km/h | "
                                  f"Distance: {distance_traveled:6.1f}m | "
                                  f"Lane error: {lane_error:+.3f}m | "
                                  f"Steering (M/F): {model_steer:+.3f}/{final_steer:+.3f}")
                            last_log_time = current_time
                    
                    frame_count += 1
                
                # Tick the simulation
                self.world.tick()

        except Exception as e:
            print(f"\nError during evaluation: {e}")
            
        finally:
            # Close log file
            log_file.close()
            
            # Create video from saved frames (exact CNN-style logic)
            try:
                images = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if images:
                    out_video = f"enhanced_evaluation_video_{int(self.start_time)}.mp4"
                    with imageio.get_writer(out_video, fps=30) as writer:
                        for img in images:
                            writer.append_data(imageio.imread(img))
                    print(f"Enhanced evaluation video saved as {out_video}")
            except Exception as e:
                print(f"Warning: failed to create video: {e}")
            finally:
                try:
                    gc.collect()
                except:
                    pass
            
            # Print final statistics
            self.print_statistics(frame_count)
            
            # Cleanup
            self.cleanup()

    def print_statistics(self, frame_count):
        """Print enhanced evaluation statistics"""
        elapsed_time = time.time() - self.start_time
        
        print("\n" + "="*60)
        print("ENHANCED EVALUATION STATISTICS")
        print("="*60)
        
        print(f"Total frames processed: {frame_count}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        print(f"Average FPS: {frame_count/elapsed_time:.2f}")
        print(f"Total collisions: {self.collision_count}")
        
        if len(self.deviations) > 0:
            print(f"\nLane keeping performance:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} meters")
            print(f"  Std deviation: {np.std(self.deviations):.3f} meters")
            print(f"  Max offset: {np.max(self.deviations):.3f} meters")
            print(f"  95th percentile: {np.percentile(self.deviations, 95):.3f} meters")
            
            # Calculate percentage of time within lane bounds
            within_lane = np.sum(np.array(self.deviations) <= 1.5) / len(self.deviations) * 100
            print(f"  Time within 1.5m of lane center: {within_lane:.1f}%")
            
        if len(self.trajectory) > 0:
            first_pos = self.trajectory[0]
            last_pos = self.trajectory[-1]
            total_distance = np.sqrt(
                (last_pos['x'] - first_pos['x'])**2 + 
                (last_pos['y'] - first_pos['y'])**2
            )
            print(f"\nTotal distance traveled: {total_distance:.2f} meters")
            
        if len(self.controls_history) > 0:
            all_speeds = [c['speed'] for c in self.controls_history]
            all_lane_errors = [c['lane_error'] for c in self.controls_history]
            
            print(f"\nSpeed statistics:")
            print(f"  Average speed: {np.mean(all_speeds)*3.6:.2f} km/h")
            print(f"  Max speed: {np.max(all_speeds)*3.6:.2f} km/h")
            
            print(f"\nLane error statistics:")
            print(f"  Mean absolute error: {np.mean(np.abs(all_lane_errors)):.3f} meters")
            print(f"  RMS error: {np.sqrt(np.mean(np.array(all_lane_errors)**2)):.3f} meters")
                
        print("="*60)
        
    def cleanup(self):
        """Clean up CARLA actors and reset settings"""
        print("\nCleaning up...")
        
        # Reset to asynchronous mode
        if hasattr(self, 'world'):
            settings = self.world.get_settings()
            settings.synchronous_mode = False
            self.world.apply_settings(settings)
        
        # Destroy actors
        actors_to_destroy = []
        if hasattr(self, 'camera'):
            try:
                self.camera.stop()
            except:
                pass
            actors_to_destroy.append(self.camera)
        if hasattr(self, 'collision_sensor'):
            try:
                self.collision_sensor.stop()
            except:
                pass
            actors_to_destroy.append(self.collision_sensor)
        if hasattr(self, 'vehicle'):
            actors_to_destroy.append(self.vehicle)
            
        for actor in actors_to_destroy:
            if actor is not None:
                try:
                    actor.destroy()
                except:
                    pass
                
        print("Cleanup complete.")


def main():
    # Path to your trained model
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    # Create evaluator and run
    evaluator = ClosedLoopEvaluator(model_path=MODEL_PATH)
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()



ENHANCED CLOSED-LOOP EVALUATION STARTED
Press Ctrl+C to stop evaluation gracefully

Spawned vehicle at: (5.63, 133.73)
Using device: cuda
Model loaded successfully from /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming up sensors...
Starting enhanced autonomous driving...

[   1.0s] Frame    30 | Speed:   0.0 km/h | Distance:    1.1m | Lane error: -0.042m | Steering (M/F): +0.024/+0.022
[   2.0s] Frame    59 | Speed:  13.4 km/h | Distance:    4.1m | Lane error: -0.068m | Steering (M/F): +0.011/+0.003
[   3.1s] Frame    89 | Speed:  13.1 km/h | Distance:    9.6m | Lane error: -0.153m | Steering (M/F): -0.012/-0.018
[   4.1s] Frame   122 | Speed:  13.1 km/h | Distance:   15.4m | Lane error: -0.128m | Steering (M/F): +0.020/+0.003
[   5.1s] Frame   156 | Speed:  13.3 km/h | Distance:   21.7m | Lane error: -0.073m | Steering (M/F): +0.001/-0.006
[   6.1s] Frame   191 | Speed:  13.7 km/h | Distance:   28.2m | Lane error: +0.015m | Steering (M/F): +0.010

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown initiated...
Enhanced evaluation video saved as enhanced_evaluation_video_1768080425.mp4

ENHANCED EVALUATION STATISTICS
Total frames processed: 2068
Total time: 75.77 seconds
Average FPS: 27.29
Total collisions: 0

Lane keeping performance:
  Average offset: 0.167 meters
  Std deviation: 0.131 meters
  Max offset: 1.424 meters
  95th percentile: 0.330 meters
  Time within 1.5m of lane center: 100.0%

Total distance traveled: 198.09 meters

Speed statistics:
  Average speed: 12.14 km/h
  Max speed: 14.27 km/h

Lane error statistics:
  Mean absolute error: 0.167 meters
  RMS error: 0.212 meters

Cleaning up...
Cleanup complete.


reslts: enhanced_eval_images_1768080221
enhanced_eval_images_1768080425

## final code- model evaluation - lane change and maintainence

In [ ]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model (unchanged) -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


# ----------------- Enhanced PID Controller -----------------
class StrongPIDController:
    """Ultra-precise lane keeping PID controller"""
    def __init__(self):
        self.Kp = 1.8  # Very strong proportional
        self.Ki = 0.08  # Moderate integral
        self.Kd = 0.45  # Strong derivative
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.8
        self.output_limit = 1.0

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Waypoint Path Follower -----------------
class WaypointFollower:
    """Follow a pre-computed waypoint path smoothly"""
    def __init__(self):
        self.path = []
        self.current_idx = 0
        self.lookahead_distance = 8.0  # meters
        
    def set_path(self, waypoints):
        """Set new waypoint path"""
        self.path = waypoints
        self.current_idx = 0
        
    def clear_path(self):
        """Clear current path"""
        self.path = []
        self.current_idx = 0
        
    def get_target_point(self, vehicle_location):
        """Get lookahead target point on path"""
        if not self.path:
            return None
            
        # Find closest waypoint
        min_dist = float('inf')
        closest_idx = self.current_idx
        
        for i in range(self.current_idx, min(len(self.path), self.current_idx + 10)):
            wp_loc = self.path[i].transform.location
            dist = math.sqrt(
                (wp_loc.x - vehicle_location.x)**2 + 
                (wp_loc.y - vehicle_location.y)**2
            )
            if dist < min_dist:
                min_dist = dist
                closest_idx = i
        
        self.current_idx = closest_idx
        
        # Find lookahead point
        target_idx = closest_idx
        cumulative_dist = 0.0
        
        for i in range(closest_idx, len(self.path) - 1):
            wp1 = self.path[i].transform.location
            wp2 = self.path[i + 1].transform.location
            segment_dist = math.sqrt(
                (wp2.x - wp1.x)**2 + (wp2.y - wp1.y)**2
            )
            cumulative_dist += segment_dist
            
            if cumulative_dist >= self.lookahead_distance:
                target_idx = i + 1
                break
        else:
            target_idx = len(self.path) - 1
            
        if target_idx >= len(self.path):
            return None
            
        return self.path[target_idx].transform.location
    
    def is_path_complete(self, vehicle_location):
        """Check if we've reached the end of path"""
        if not self.path:
            return True
        
        end_loc = self.path[-1].transform.location
        dist = math.sqrt(
            (end_loc.x - vehicle_location.x)**2 + 
            (end_loc.y - vehicle_location.y)**2
        )
        return dist < 3.0


# ----------------- Main Controller -----------------
class ImprovedLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Controllers
        self.lane_pid = StrongPIDController()
        self.path_follower = WaypointFollower()
        
        # Smoothing
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.25  # Smoother steering
        self.throttle_alpha = 0.20

        # Speed control
        self.target_speed_kmh = 20.0
        self.stop_distance_from_line = 3.5  # Stop 3.5m before traffic light

        # State machine
        self.state = "DRIVING"
        self.stop_start_time = None
        self.green_wait_time = 0.0
        self.lane_change_decision = None  # "left" or "right"
        self.lane_change_path_created = False

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0

        # CARLA objects
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None
        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nShutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        
        blueprint_library = self.world.get_blueprint_library()
        vehicle_bp = blueprint_library.filter("vehicle.mercedes.coupe_2020")[0]
        if not vehicle_bp:
            vehicle_bp = blueprint_library.filter("vehicle.*")[0]
        # bp_lib = self.world.get_blueprint_library()
        # vehicle_bp = bp_lib.filter("vehicle.tesla.model3")[0] if bp_lib.filter("vehicle.tesla.model3") else random.choice(bp_lib.filter("vehicle.*"))
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        # Camera
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "120")
        cam_transform = carla.Transform(carla.Location(x=-4,y=0, z=2), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        # Collision sensor
        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (800, 600))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.collision_occurred = True
        
        # Emergency stop and reset
        self.path_follower.clear_path()
        self.lane_pid.reset()
        self.state = "COLLISION_RECOVERY"
        
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        
        time.sleep(1.5)
        self.collision_occurred = False
        self.state = "DRIVING"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("Model loaded")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate cross-track and heading error"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            # Cross-track error
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Heading error
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combined error with strong heading weight
            total_error = cross_track + 1.2 * heading_error
            return total_error
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        """Find nearest traffic light ahead of vehicle"""
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 40.0 or dist < 0.5:
                    continue
                
                # Check if ahead
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:  # Ahead of us
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_availability(self, current_wp, direction):
        """Check if lane change is safe and valid"""
        try:
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return False
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Must be same direction (check lane_id sign)
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            # Check path ahead is valid
            test_wp = target_wp
            valid_distance = 0.0
            for _ in range(30):
                nxts = test_wp.next(2.0)
                if not nxts:
                    break
                test_wp = nxts[0]
                if test_wp.lane_type != carla.LaneType.Driving:
                    break
                valid_distance += 2.0
            
            if valid_distance < 40.0:  # Need good path ahead
                return False
            
            # Check for vehicles in target lane
            veh_loc = self.vehicle.get_location()
            for other in self.world.get_actors().filter("vehicle.*"):
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dist = math.sqrt(
                    (other_loc.x - veh_loc.x)**2 + 
                    (other_loc.y - veh_loc.y)**2
                )
                
                if dist < 25.0:  # Check nearby vehicles
                    try:
                        other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                        if other_wp and other_wp.lane_id == target_wp.lane_id:
                            return False  # Vehicle in target lane
                    except Exception:
                        pass
            
            return True
            
        except Exception as e:
            print(f"Lane check error: {e}")
            return False

    def create_lane_change_path(self, direction):
        """Create smooth waypoint path for lane change"""
        try:
            loc = self.vehicle.get_location()
            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if current_wp is None:
                return []
            
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return []
            
            # Build path: current -> transition -> target
            path = []
            
            # Phase 1: Move forward in current lane (10m)
            wp = current_wp
            for _ in range(5):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            # Phase 2: Transition to target lane (20m smooth curve)
            start_wp = wp
            
            # Get corresponding point in target lane ahead
            end_wp = target_wp
            for _ in range(15):
                nxts = end_wp.next(2.0)
                if nxts:
                    end_wp = nxts[0]
            
            # Create interpolated waypoints
            for i in range(1, 11):
                t = i / 10.0
                # Smooth interpolation
                interp_loc = carla.Location(
                    x=start_wp.transform.location.x * (1 - t) + end_wp.transform.location.x * t,
                    y=start_wp.transform.location.y * (1 - t) + end_wp.transform.location.y * t,
                    z=start_wp.transform.location.z * (1 - t) + end_wp.transform.location.z * t
                )
                interp_wp = self.map.get_waypoint(interp_loc, project_to_road=True)
                if interp_wp:
                    path.append(interp_wp)
            
            # Phase 3: Continue in target lane (20m)
            wp = end_wp
            for _ in range(10):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            return path
            
        except Exception as e:
            print(f"Path creation error: {e}")
            return []

    def calculate_steering_to_point(self, vehicle_loc, vehicle_rot, target_point):
        """Pure pursuit steering to target point"""
        # Vector to target
        dx = target_point.x - vehicle_loc.x
        dy = target_point.y - vehicle_loc.y
        
        # Transform to vehicle frame
        yaw = math.radians(vehicle_rot.yaw)
        local_x = dx * math.cos(yaw) + dy * math.sin(yaw)
        local_y = -dx * math.sin(yaw) + dy * math.cos(yaw)
        
        # Steering angle
        if abs(local_x) < 0.1:
            return 0.0
        
        steering_angle = math.atan2(local_y, local_x)
        
        # Normalize to [-1, 1]
        steer_cmd = steering_angle / math.radians(30)  # Max 30 degrees
        return float(np.clip(steer_cmd, -1.0, 1.0))

    def adaptive_speed_control(self, current_speed_mps, lane_error, steering_mag):
        """Adaptive throttle based on conditions"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed with lane error
        lane_factor = max(0.7, 1.0 - abs(lane_error) / 3.0)
        
        # Reduce speed with steering
        steer_factor = max(0.75, 1.0 - abs(steering_mag) / 0.6)
        
        adjusted_target = target_speed_mps * lane_factor * steer_factor
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.0, 0.35 + speed_error * 0.25)
        
        return float(np.clip(throttle, 0.0, 0.8))

    def run_evaluation(self):
        print("\n" + "="*70)
        print("SMOOTH LANE CHANGE WITH PRECISE STOPPING")
        print("="*70)
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        # Logging
        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_kmh,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                # Update spectator
                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Get traffic light info
                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control loop
                if len(frame_stack) == 3:
                    # Get vehicle state
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    # Lane error
                    lane_error = self.calculate_lane_error(loc, rot)

                    # Get model prediction
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    # Initialize control
                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0

                    # ============ STATE MACHINE ============
                    
                    if self.state == "DRIVING":
                        # Check for red light
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 25.0):
                            print(f"🔴 Red light @ {tl_dist:.1f}m -> STOPPING")
                            self.state = "APPROACHING_STOP"
                            self.stop_start_time = time.time()
                        else:
                            # Normal lane keeping with strong PID
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            # Blend model and PID (heavy PID weight)
                            error_weight = min(abs(lane_error) / 0.5, 0.90)
                            combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * combined_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.7, 0.7))
                            
                            # Adaptive throttle
                            final_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                            final_brake = 0.0
                    
                    elif self.state == "APPROACHING_STOP":
                        # Smooth braking to stop at correct position
                        if tl_dist is not None:
                            # Calculate brake based on distance and speed
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.5 or speed_mps < 0.3:
                                # Arrived at stop line
                                print("🛑 Stopped at line, waiting for green...")
                                self.state = "WAITING_GREEN"
                                self.green_wait_time = 0.0
                                self.lane_pid.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                # Proportional braking
                                brake_strength = min(1.0, 0.4 + (speed_mps / 15.0) + (1.0 / max(stop_dist, 1.0)))
                                final_brake = float(np.clip(brake_strength, 0.3, 1.0))
                                final_throttle = 0.0
                                
                                # Gentle steering to maintain lane while braking
                                now = time.time()
                                pid_correction = self.lane_pid.update(lane_error, now)
                                final_steer = 0.5 * pid_correction  # Gentle correction
                                final_steer = float(np.clip(final_steer, -0.3, 0.3))
                        else:
                            # Lost traffic light, stop anyway
                            final_brake = 0.8
                            final_throttle = 0.0
                            final_steer = 0.0
                    
                    elif self.state == "WAITING_GREEN":
                        # Wait at stop line
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        # Check for green light
                        if tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                            self.green_wait_time += 0.05  # One tick
                            
                            if self.green_wait_time > 0.5:  # Wait 0.5s after green
                                print("🟢 Green light -> Deciding lane change...")
                                
                                # Decide lane change direction
                                loc = self.vehicle.get_location()
                                current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                                
                                if current_wp:
                                    # Try random direction first
                                    directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
                                    
                                    for direction in directions:
                                        if self.check_lane_availability(current_wp, direction):
                                            print(f"  ✓ {direction.upper()} lane is available")
                                            self.lane_change_decision = direction
                                            self.state = "INITIATING_LANE_CHANGE"
                                            break
                                    else:
                                        print("  ✗ No safe lane, going straight")
                                        self.state = "DRIVING"
                                        self.lane_pid.reset()
                                else:
                                    print("  ✗ Cannot get waypoint, going straight")
                                    self.state = "DRIVING"
                                    self.lane_pid.reset()
                    
                    elif self.state == "INITIATING_LANE_CHANGE":
                        # Create lane change path
                        if not self.lane_change_path_created:
                            path = self.create_lane_change_path(self.lane_change_decision)
                            
                            if path:
                                self.path_follower.set_path(path)
                                self.lane_change_path_created = True
                                print(f"  🔄 Lane change path created ({len(path)} waypoints)")
                                self.state = "EXECUTING_LANE_CHANGE"
                            else:
                                print("  ✗ Failed to create path, aborting")
                                self.state = "DRIVING"
                                self.lane_pid.reset()
                        
                        # While creating, stay stopped
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                    
                    elif self.state == "EXECUTING_LANE_CHANGE":
                        # Follow the waypoint path
                        target_point = self.path_follower.get_target_point(loc)
                        
                        if target_point is not None:
                            # Calculate steering to target
                            final_steer = self.calculate_steering_to_point(loc, rot, target_point)
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * final_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.6, 0.6))
                            
                            # Moderate speed during lane change
                            target_lc_speed = 15.0  # km/h
                            lc_speed_mps = target_lc_speed / 3.6
                            speed_error = lc_speed_mps - speed_mps
                            
                            if speed_error > 0:
                                final_throttle = min(0.5, 0.3 + speed_error * 0.2)
                            else:
                                final_throttle = 0.2
                            
                            final_brake = 0.0
                            
                            # Check if path complete
                            if self.path_follower.is_path_complete(loc):
                                print(f"  ✅ Lane change to {self.lane_change_decision.upper()} COMPLETE!")
                                self.lane_changes_completed += 1
                                self.path_follower.clear_path()
                                self.lane_change_path_created = False
                                self.lane_change_decision = None
                                self.state = "STABILIZING"
                                self.stop_start_time = time.time()
                        else:
                            # Lost path, abort
                            print("  ⚠️ Lost path target, aborting")
                            self.path_follower.clear_path()
                            self.lane_change_path_created = False
                            self.lane_change_decision = None
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                            final_steer = 0.0
                            final_throttle = 0.0
                            final_brake = 0.3
                    
                    elif self.state == "STABILIZING":
                        # After lane change, stabilize in new lane
                        elapsed = time.time() - self.stop_start_time
                        
                        if elapsed < 2.0:
                            # Strong PID to center in lane
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            final_steer = pid_correction
                            final_steer = float(np.clip(final_steer, -0.5, 0.5))
                            
                            # Gentle acceleration
                            final_throttle = min(0.4, 0.15 + elapsed * 0.1)
                            final_brake = 0.0
                        else:
                            print("  📍 Stabilized in new lane, resuming normal driving")
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                    
                    elif self.state == "COLLISION_RECOVERY":
                        # Recovery from collision
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0

                    # ============ APPLY CONTROL ============
                    
                    # Apply smoothing to steering
                    self.steer_smooth = final_steer
                    
                    # Apply smoothing to throttle
                    self.throttle_smooth = final_throttle
                    
                    # Apply to vehicle
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # ============ LOGGING & RECORDING ============
                    
                    # Save frame
                    # try:
                    #     if self._saved_frame_ema is not None:
                    #         save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                    #         save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                    #         cv2.imwrite(save_path, save_img)
                    # except Exception:
                    #     pass
                    try:
                            frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(frame_filename, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                    except Exception as e:
                            print(f"Warning: failed to save frame {frame_count}: {e}")

                    # Statistics
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    # Log to CSV
                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},"
                        f"{rot.yaw:.2f},{speed_kmh:.1f},{final_steer:.4f},{final_throttle:.4f},"
                        f"{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    # Console output
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name[:3]} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@ {tl_dist:.1f}m"
                        
                        lc_info = f"[LC: {self.lane_changes_completed}]" if self.lane_changes_completed > 0 else ""
                        
                        print(
                            f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                            f"Speed:{speed_kmh:5.1f} | "
                            f"Lane:{lane_error:+.3f}m | "
                            f"Steer:{final_steer:+.3f} | "
                            f"TL:{tl_msg:15s} | "
                            f"{self.state:22s} {lc_info}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            # Create video
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"\nCreating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Video saved:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("EVALUATION STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f} s")
        print(f"Frames: {frame_count}")
        if elapsed > 0:
            print(f"FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Changes:")
        print(f"  Completed: {self.lane_changes_completed}")
        
        if self.deviations:
            print(f"\nLane Keeping:")
            print(f"  Mean error: {np.mean(self.deviations):.3f} m")
            print(f"  Max error: {np.max(self.deviations):.3f} m")
            print(f"  Std dev: {np.std(self.deviations):.3f} m")
            within_half = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            within_quarter = sum(1 for d in self.deviations if d < 0.25) / len(self.deviations) * 100
            print(f"  Within 0.5m: {within_half:.1f}%")
            print(f"  Within 0.25m: {within_quarter:.1f}%")
        
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\nDistance traveled: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed = (total_dist / elapsed) * 3.6
                print(f"Average speed: {avg_speed:.2f} km/h")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                except Exception:
                    pass
                actors.append(a)
        
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        
        gc.collect()
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = ImprovedLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


SMOOTH LANE CHANGE WITH PRECISE STOPPING
Spawned at (27.14, 66.28)
Device: cuda
Model loaded
Warming up sensors...
[   1.0s] F  30 | Speed:  0.0 | Lane:-0.000m | Steer:+0.001 | TL:None            | DRIVING                
[   2.1s] F  63 | Speed: 17.5 | Lane:+0.044m | Steer:+0.007 | TL:None            | DRIVING                
[   3.1s] F  96 | Speed: 18.6 | Lane:-0.052m | Steer:-0.021 | TL:None            | DRIVING                
[   4.1s] F 129 | Speed: 18.4 | Lane:+0.044m | Steer:+0.019 | TL:None            | DRIVING                
[   5.1s] F 161 | Speed: 18.4 | Lane:-0.074m | Steer:-0.016 | TL:None            | DRIVING                
[   6.1s] F 193 | Speed: 18.2 | Lane:+0.084m | Steer:+0.035 | TL:None            | DRIVING                
[   7.1s] F 226 | Speed: 17.6 | Lane:-0.179m | Steer:-0.126 | TL:None            | DRIVING                
[   8.1s] F 259 | Speed: 15.3 | Lane:+0.161m | Steer:+0.110 | TL:None            | DRIVING                
[   9.2s] F 293 | Speed: 18.